<center><img src="https://github.com/DACSS-Spatial/data_forSpatial/raw/main/logo.png" width="700"></center>

<a target="_blank" href="https://colab.research.google.com/github/DACSS-Spatial/TheGeo-DataFrame/blob/main/index.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# The Geo Dataframe

The geodataframe (GDF) is a data table (a dataframe or DF) where every row represents an spatial element (point, line, polygon). In Python, to work on DF (simple tables) you need external packages (**pandas**), and for GDF you also need an external package. We will use **geopandas**.

You have some maps in zipped files for this session:

1.   World countries, rivers, and cities.
2.   Brazil regions and municipalities.

You also have a data table (DF): Airports of Brazil.

## Getting ready for the session

Please create a _repo_ for this session on _GitHub_ (name it as you think convenient). Once you clone the repo to your local machine, put all those GDF and DF in the repo and them _push_ them to the GitHub cloud. You should follow this session using your own _urls_ (you will use your own, replacing mine).

Yoy MUST read your DFs or GDFs from GitHub.

## The First GDF

With the help of **geopandas** and GitHub, we will bring the world countries:

In [ ]:
import geopandas as gpd

linkGitSession='https://github.com/DACSS-Spatial/data_forSpatial/raw/refs/heads/main/'

linkCountries='WORLD/World_Countries.zip'

fullLinkCountries=linkGitSession+linkCountries
countries=gpd.read_file(fullLinkCountries)

If no error was shown, we have the GDF. Let's visit it:

In [ ]:
# what is it?
type(countries)

In [ ]:
# dimensions
countries.shape

In [ ]:
# names
countries.columns

In [ ]:
# any missing values?
countries[countries.isna().any(axis=1)]

In [ ]:
# data types in the GDF
countries.info()

In [ ]:
# some content
countries.head()

Al the previous commands exist and work the same if you had a DF from pandas. But now we have a new column type: **geometry**, and we can have some extra functiond from _geopandas_:

In [ ]:
# what geometry?
countries.geom_type.unique()

As we have polygons, we know the map is about country borders:

In [ ]:
countries.plot(facecolor="azure",#color of polygon fill
               edgecolor='red', #color of border
               linewidth=0.1) #thickness of lines

## More GDFs

Let's open the other maps:

In [ ]:
linkRivers='WORLD/World_Hydrography.zip'
linkCities='WORLD/World_Cities.zip'

fullLinkRivers=linkGitSession+linkRivers
fullLinkCities=linkGitSession+linkCities


rivers=gpd.read_file(fullLinkRivers)
cities=gpd.read_file(fullLinkCities)

This is the rivers map:

In [ ]:
rivers.plot(edgecolor='blue',
            linewidth=1,
            linestyle='-')

This is the cities map:

In [ ]:
cities.plot(marker='.', # marker type
            color='red',
            markersize=1,
            alpha=0.3) # transparency

As we can see above, the rivers was a map of **lines**, while the cities were a map of **points**.

You can start by creating the **base** layer, and add other maps, as layers, on top:

In [ ]:
base = countries.plot(facecolor="white",
                      edgecolor='black',
                      linewidth=0.1,
                      figsize=(12,12))

rivers.plot(edgecolor='blue', linewidth=0.4,
            ax=base)# on top of...
cities.plot(marker='.', color='red', markersize=1,alpha=0.7,
            ax=base) # on top of...



# The Map Projection

We know the reality is at least in 3D, so on this screen and on piece of paper we should alter reality so the earth or parts of it can be seen in a meaningful way.

Let's find out about the way the maps we have are represented in 2D:

In [ ]:
countries.crs

In [ ]:
cities.crs

In [ ]:
rivers.crs

In all the previous cases, you see a common CRS (EPSG 4326), and also that their axes are in **long** / **lat** (measured in *degrees*). That is a clear sign the the map is **NOT projected**.

At this point, let me keep one country:

In [ ]:
brazil=countries[countries.COUNTRY=='Brazil']

The country (or the subset of the countries map) will inherit the original projection:

In [ ]:
brazil.crs

In [ ]:
# check units of measurement
brazil.crs.axis_info

In [ ]:
# is this CRS projected?
brazil.crs.is_projected

The fact it is not projected, means that no operation involving distances gives reliable nor consistente results. For instance, we know Polygons have a centroid; but when we try getting a centroid from an **unprojected** polygon, you get:

In [ ]:
# a warning from python!
brazil.centroid

In [ ]:
# this does not look bad
base=brazil.plot()
brazil.centroid.plot(ax=base,color='red')

## Reprojecting

A projected CRS will have units in meters or feet (or similar). You can request a EPSG per country [here](https://epsg.io/?q=brazil+kind%3APROJCRS), but also debating with AI is good alternative.

In [ ]:
# recommended from website
brazil.to_crs(5641).crs.axis_info

In [ ]:
# now this works with no warning
brazil.to_crs(5641).centroid

In [ ]:
# recommended from Gemini - after a long chat!
brazil.to_crs(5880).crs.axis_info

In [ ]:
# no warning either!
brazil.to_crs(5880).centroid

Let's plot!

In [ ]:
# replotting:

base5880=brazil.to_crs(5880).plot()
brazil.to_crs(5641).centroid.plot(color='black',ax=base5880)
brazil.to_crs(5880).centroid.plot(color='lightgreen',ax=base5880)
brazil.centroid.plot(color='red',ax=base5880)

Let's keep the **EPSG: 5880**  for the the country and cities:

In [ ]:
brazil_5880=brazil.to_crs(5880)
cities_brazil_5880=cities[cities.COUNTRY=='Brazil'].to_crs(5880)

### Think about...
<div class="alert-success">

What would be a good CRS for your country, state, city?
    
</div>

## Creating a spatial dataframe

You have a CSV file with information on the airports in Brazil. Save it to GitHub and open it:

In [ ]:
import pandas as pd #using pandas

infoairports=pd.read_csv(linkGitSession+'BRAZIL/br-airports.csv')

# take a look

infoairports #head and tail


Notice the firt row must not stay in the data. Then,

In [ ]:
# bye first row
infoairports.drop(index=0, #bye row 'zero'
                  inplace=True) #immediately

infoairports.reset_index(drop=True, # reset the index (row numbers)
                         inplace=True) # immediately
infoairports.head()

This DF has so many columns:

In [ ]:
infoairports.columns


In [ ]:
# keeping some

# list of names to keep
keep=['name','type','latitude_deg', 'longitude_deg','elevation_ft','region_name','municipality']

infoairports=infoairports.loc[:,keep] # 'loc' when you have column names

# less columns.

infoairports.info() # checking data types

We have two problems here:

*   A column name is **type**. You must not use that name in a pandas dataframe column. Optionally change 'name' to 'airport_name'.
*   Columns 2,3 and 4 are 'object', and they should be numeric. Then, we should use pandas' **pd.to_numeric()** function:

Solving problem 1:

In [ ]:
infoairports.rename(columns={'type':'airport_type','name':'airport_name'},inplace=True)

Solving problem 2:

In [ ]:
ColumnsToNumeric=['latitude_deg', 'longitude_deg','elevation_ft']

infoairports[ColumnsToNumeric]=infoairports.loc[:,ColumnsToNumeric].apply(lambda x:pd.to_numeric(x))

In [ ]:
# RE verifying
infoairports.info()

In [ ]:
# let's plot using the unprojected map as base:

base = brazil.plot(color='white', edgecolor='black') #unprojected

infoairports.plot.scatter(x = 'longitude_deg', y = 'latitude_deg',ax=base)

The plotting worked well, but this is a poor choice.

### Think about...
<div class="alert-success">

Why this would be wrong?
    
</div>

Let's use the projected BRAZIL:

In [ ]:
base = brazil_5880.plot(color='white', edgecolor='black')

infoairports.plot.scatter(x = 'longitude_deg', y = 'latitude_deg',ax=base)

That results only shows the different coordinate systems. But a bigger problem may be that the airports are not a spatial object:

In [ ]:
type(infoairports)

The job is to turn the DF info a GDF:

In [ ]:
airports_gdf=gpd.GeoDataFrame(data=infoairports,
                          geometry=gpd.points_from_xy(infoairports.longitude_deg,
                                                      infoairports.latitude_deg),
                          crs=brazil.crs.to_epsg())# the original coordinates were unprojected!


In [ ]:
#verify:
type(airports_gdf)

Now that we have a GDF, we can reproject:

In [ ]:
airports_gdf_5880=airports_gdf.to_crs(5880)

## then

base = brazil_5880.plot(color='white', edgecolor='black')
airports_gdf_5880.plot(ax=base)

<a class="anchor" id="11"></a>

## Formating Spatial objects into GeoDataFrames



You know **cities_brazil_5880** is a GDF:

In [ ]:
type(cities_brazil_5880)

Is the brazil centroid a dataframe?

In [ ]:
brazil_5880.centroid

In [ ]:
type(brazil_5880.centroid)

You are getting a **GeoSeries** (GS).

In case an operation in Geopandas required a GDF instead of a GS, you may need this code:

In [ ]:
brazil_5880.centroid.to_frame()

We can give a better look to this GDF:

In [ ]:
brazil_5880.centroid.to_frame().info()

Notice you get a very simple GDF, and you may want to add some information:

In [ ]:
# conversion
centroids=brazil_5880.centroid.to_frame()

# new column (optional)
centroids['country']='Brazil'

# renaming the geometry column
centroids.rename(columns={0:'geometry'},inplace=True)

#setting the geometry (the name is not enough)
centroids = centroids.set_geometry("geometry")

# verifying:
centroids.crs

In [ ]:
centroids

<a class="anchor" id="12"></a>

## Maps Lacking CRS information

Reprojecting seems a simple process, but you might find some interesting cases.

Let's read the maps on _states_(adm1) and _municipalities_ (adm2) on Brazil to find some interesting facts:

In [ ]:
brazil_states=gpd.read_file(linkGitSession+'BRAZIL/bra_admbnda_adm1_ibge_2020.zip')
brazil_municipalities=gpd.read_file(linkGitSession+'BRAZIL/bra_admbnda_adm2_ibge_2020.zip')

They are maps, for sure:

In [ ]:
type(brazil_states), type(brazil_municipalities)

But, notice this:

In [ ]:
brazil_states.crs, brazil_municipalities.crs

They do not have crs information, however they can be plotted:

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=False, sharey=False, figsize=(12,12))

brazil_states.plot(ax=ax1, facecolor='lightgrey', edgecolor='black')
brazil_municipalities.plot(ax=ax2, facecolor='lightgrey', edgecolor='black',linewidth=0.2)


Since we are using the crs 5880 for Brazil, the initial strategy could be to set the CRS with the right projection :

In [ ]:
## uncomment this to see the error message
# brazil_states.to_crs(5880)

Python says _"Please set a crs on the object first"_. This would mean to know the original projection of the geometry:

From the plots above and the axes shown, we conclude the maps are unprojected, then:

In [ ]:
# set as unprojected
brazil_states.crs = "EPSG:4326"
brazil_municipalities.crs = "EPSG:4326"

Now, we can reproject:

In [ ]:
brazil_states_5880=brazil_states.to_crs(5880)
brazil_municipalities_5880=brazil_municipalities.to_crs(5880)

# Saving

We will create a file readabel by R or Python, in this case a GeoPackage file (**.gpkg**). This file allows you to save all the previous maps as **layers** in **one** file.

In [ ]:
## Good practice to refresh your memory

## Find only GeoDataFrames in the current session
geodataframe_list = [var for var in globals() if isinstance(globals()[var], gpd.GeoDataFrame)]

print("List of GeoDataFrames in memory:")
for name in geodataframe_list:
    print(name)

We know we will save all the Brazil's 5880:
brazil_5880,cities_brazil_5880, airports_gdf_5880, brazil_states_5880, brazil_municipalities_5880.
...and also, from 'world': countries,rivers, and cities.

Let's check those column names are more intuitive:

In [ ]:
brazil_5880.columns

In [ ]:
cities_brazil_5880.columns

In [ ]:
airports_gdf_5880.columns

In [ ]:
brazil_states_5880.columns

In [ ]:
brazil_municipalities_5880.columns

Let's drop and rename some columns in the last three:

In [ ]:
airports_gdf_5880.drop(columns=['latitude_deg', 'longitude_deg'],inplace=True)
airports_gdf_5880.rename(columns={'region_name':'region'},inplace=True)

brazil_states_5880.drop(columns=['ADM0_EN', 'ADM0_PT', 'ADM0_PCODE'],inplace=True)
brazil_states_5880.rename(columns={'ADM1_PT':'state_name','ADM1_PCODE':'state_code'},inplace=True)

brazil_municipalities_5880.drop(columns=['ADM0_EN', 'ADM0_PT', 'ADM0_PCODE','ET_ID'],inplace=True)
brazil_municipalities_5880.rename(columns={'ADM1_PT':'state_name','ADM1_PCODE':'state_code','ADM2_PT':"municipality_name",
       'ADM2_PCODE':"municipality_code"},inplace=True)


We choose what to save for later use (send it to GitHub):

In [ ]:
### from the list above:
brazil_5880.to_file('brazil_5880.gpkg',driver='GPKG',layer='country')
cities_brazil_5880.to_file('brazil_5880.gpkg',driver='GPKG',layer='cities')
airports_gdf_5880.to_file('brazil_5880.gpkg',driver='GPKG',layer='airports')
brazil_states_5880.to_file('brazil_5880.gpkg',driver='GPKG',layer='states')
brazil_municipalities_5880.to_file('brazil_5880.gpkg',driver='GPKG',layer='municipalities')

Let's check the names of the other group:

In [ ]:
rivers.columns

In [ ]:
cities.columns

In [ ]:
countries.columns

Let's rename:

In [ ]:
rivers.rename(columns={'NAME':'RIVER'},inplace=True)
cities.rename(columns={'NAME':'CITY'},inplace=True)

We may also create:

In [ ]:
rivers.to_file('worldMaps.gpkg',driver='GPKG',layer='rivers')
cities.to_file('worldMaps.gpkg',driver='GPKG',layer='cities')
countries.to_file('worldMaps.gpkg',driver='GPKG',layer='countries')